In [ ]:
# Implementar algoritmo de agrupación (por ejemplo, K-Means)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import pandas as pd 

In [ ]:
df = pd.read_csv()

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['titulo_video'])
num_clusters = 10  # Número de clusters (temáticas)
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(tfidf_matrix)

# Agregar columna de cluster al DataFrame
df['cluster'] = kmeans.labels_

In [ ]:
df.cluster.value_counts()

with spacy 

In [ ]:
import spacy

# Cargar modelo de lenguaje spaCy
# es_core_news_lg
# en_core_web_sm
nlp = spacy.load("en_core_web_lg")

# Crear un knowledge graph
knowledge_graph = {}

for index, row in df.iterrows():
    doc = nlp(row['titulo_video'])
    keywords = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]
    for keyword in keywords:
        if keyword not in knowledge_graph:
            knowledge_graph[keyword] = set()
        knowledge_graph[keyword].add(row['titulo_video'])

# Imprimir el knowledge graph
for keyword, related_videos in knowledge_graph.items():
    print(f"{keyword}: {related_videos}")